In [20]:
from setuptools import setup
from setuptools import Extension

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [21]:
# import packages we need
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from os import walk
import time
import calendar
import datetime

In [22]:
from bs4 import BeautifulSoup

In [1]:
import os
import codecs
import re

# Welt Data (1999-2018)

In [2]:
%%time
path = 'E:\\Userhome\\mokuneva\\Welt' #your path here

journal = []
day = []
month = []
year = []
titles = []
texts = []
section = []

# os.listdir(path) - names of directories: '1999', '2000'...
# we proceed folder by folder
for f in [f for f in os.listdir(path) ]: 
    # again folder by folder
    for s in [s for s in os.listdir(path + '\\' + f) ]:
        #for each file in a particular folder
        for k in [k for k in os.listdir(path +  '\\' + f + '\\' + s) ]: 
            with codecs.open(path + '\\' + f + '\\' + s + '\\' + k, "r", encoding = 'utf-8' ) as input_data:
                for line in input_data:
                    # the line containing "Download" looks like this: "Download Request: Select Items: 601-642"
                    # repeat: the number of articles in the document, here 642-601+1
                    if line.find('Download') != -1:   
                        repeat = int(line.split()[4].split('-')[1])-int(line.split()[4].split('-')[0])+1
                        break
                    # documents from 2018: "Ausgabeauftrag: Dokument(e): 1-200"
                    # this means that a search was completed in German
                    elif line.find('Ausgabeauftrag') != -1:
                        repeat = int(line.split()[2].split('-')[1])-int(line.split()[2].split('-')[0])+1
                        break
                # for each article in the document
                for i in range(repeat):
                    text = []
                    # Skips text before the beginning of the interesting block:
                    for line in input_data:
                        # find the line including the string "DOCUMENTS", skip the text before this line
                        if ((line.find('DOCUMENTS') != -1)|(line.find('Dokument') != -1)) :   #
                            break
                            
                    # NESPAPER NAME (makes sense to add as the data is coming from several sources) 
                    for line in input_data:
                        if ((line.find('Die Welt') != -1) | (line.find('DIE WELT') != -1)) :
                            journal.append('Welt')
                            break
                    
                    # DAY/MONTH/YEAR
                    for line in input_data:
                        if line.find('.') != -1:
                            if len(line.split()) == 3:
                                day.append(line.split()[0][:-1])
                                month.append(line.split()[1])
                                year.append(line.split()[2])
                                break
                            else:
                                day.append(line.split()[1][:-1])
                                month.append(line.split()[2])
                                year.append(line.split()[3])
                                break
                    
                    # TITLE AND ANNOTATION
                    title = []
                    for line in input_data:
                        if (line.find('RUBRIK') != -1):
                            # in the title:
                            # 1. strip whitespace on both sides
                            # 2. remove tab and line break
                            title[1] = title[1].strip().replace("\n", '').replace("\t", '').replace("\r", '')
                            # if the title ends with ";", remove it
                            title[1] = title[1].replace(";", '') 
                            # if there is no period, colon, semicolon, exclamation, or question mark
                            # at the end of the sentence, add the period.
                            if title[1][-1] not in ['.', '!', ':', ';', '?']:
                                title[1] = title[1] + '.' 
                                
                            title = ' '.join(title)
                            # If the section contains 'kompakt', then the text already contains the titles
                            # of subsections, and annotation is not required.
                            if ((line.find('kompakt') == -1) & (line.find('Kompakt') == -1)):
                                title = title.strip().replace("\n", '').replace("\t", '').replace("\r", '')
                                # replace ' ++' in the annotation with '.'
                                title = title.replace(' ++', '.')
                                if title[-1] not in ['.', '!', ':', ';', '?', '"']: 
                                    title = title + '.'
                                titles.append(title)
                            else:
                                titles.append('')
                            section.append(line.split(':')[1].split(';')[0].strip())
                            break
                        elif (line.find('AUTOR') == -1):
                            title.append(line)
                    
                    # TEXT
                    for line in input_data:
                        if line.find(u'Wörter') != -1:
                            break
                    for line in input_data:
                        if line.find('UPDATE') != -1:
                            break
                        line = line.strip()
                        line = line.replace("\n", '').replace("\r", '').replace("\t", '')
                        if line != '':
                            if ((line[-1] not in ['.', '!', ':', ';', '?', '"', "'"]) & (len(line.split(' '))<6)):
                                line = line + '.'
                            line = line.replace('Weitere Informationen im Internet unter: ', '')
                            # remove URLs
                            line = re.sub(r'https\S+', '', line)
                            line = re.sub(r'http\S+', '', line)
                            line = re.sub(r'www.\S+', '', line)
                            # replace a non-breaking space with a space
                            line = re.sub('\xa0', ' ', line)
                            text.append(line)      
                    text = ' '.join(text)
                    text = text.strip()
                    # if the text is not empty, then merge the title, annotation and the text:
                    if ((len(text)>0) & (len(titles[-1])>0)):
                        text = titles[-1] + ' ' + text
                    texts.append(text)

Wall time: 1min 53s


In [3]:
# create dictionary to transform month name into month number
name_to_number = {u'Januar':1, u'Februar':2, u'M\xe4rz':3, u'April':4, u'Mai':5, u'Juni':6, u'Juli':7, u'August':8, u'September':9, u'Oktober':10, 
                  u'November':11, u'Dezember':12}

In [4]:
# month name to month number
month_num = []
for m in month:
    month_num.append(name_to_number[m])

In [5]:
# day to integer, year to integer
day = list(map(int, day))
year = list(map(int, year))

In [6]:
# now we will create a DataFrame 
import pandas as pd
data = pd.DataFrame({'year' : year,
 'month' : month_num,
 'day' : day,
 'journal': journal,
 'texts': texts,
 'section': section,
 'titles': titles})

# Light pre-processing

In [7]:
# remove the articles with an empty text (normally these are descriptions of the pictures)
data = data[data['texts'] != '']
data = data.reset_index()
del data['index']

In [8]:
# sorting values
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
data = data.reset_index()
del data['index']

## Duplicates

In [9]:
welt_duplicates = data[data['texts'].isin(data['texts'][data['texts'].duplicated()])]

Possible reasons for the existence of duplicated articles:
* The same article was published in different sections: e.g. in 'Wirtschaft kompakt' (general) and 'Berlin Wirtschaft' (region-specific).
* The series of news (e.g. about taxes) is introduced several times.
* Simple repetition without any good reason.

In [11]:
# drop duplicates
data = data.drop_duplicates(['texts'], keep = 'first')
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [139]:
# Save the data after very simple pre-processing
data.to_csv('cleaned_welt.csv', encoding='utf-8-sig', sep = ';')

# Filtering

## Sections

In [12]:
# Sections and the number of articles per section
import collections
counter=collections.Counter(data['section'])
print(counter.most_common(1000))

[('WIRTSCHAFT', 69791), ('Wirtschaft', 63652), ('FINANZEN', 27078), ('Finanzen', 24209), ('Hamburg Wirtschaft', 7186), ('Wirtschaft kompakt', 1106), ('Berlin Wirtschaft', 1055), ('Wirtschaft Kompakt', 693), ('Finanzen kompakt', 159), ('Finanzen Kompakt', 67), ('Extra', 39), ('Extra Messe-Wirtschaft', 27), ('Berlin', 27), ('Berliner Web Wirtschaft', 26), ('FINANZEN Ausg. 241', 24), ('HH-WIRTSCHAFT', 22), ('FINANZEN Ausg. 228', 19), ('KLIMA & FINANZEN Ausg. 19', 17), ('FINANZEN Ausg. 105', 17), ('Hamburg', 13), ('FINANZEN Ausg. 115', 11), ('Berliner Wirtschaft', 10), ('Wissenschaft', 9), ('FINANZEN Ausg. 90', 8), ('Wirtschafts-Bücher', 8), ('WIRTSCHAFT GESAMT', 6), ('Berlin Werbe Wirtschaft', 6), ('BERLIN WIRTSCHAFT', 6), ('WebWirtschaft', 6), ('wirtschaft kompakt', 6), ('HAMBURG', 5), ('Messe-Wirtschaft', 4), ('Wirtschafts Bücher', 4), ('Sonderseite Messe Wirtschaft', 4), ('wirtschaft-bestseller', 3), ('WISSENSCHAFT', 3), ('finanzen kompakt', 3), ('FINANZEN Ausg. 76', 3), ('Sonderseite 

In [13]:
keep_sections = ['WIRTSCHAFT', 'Wirtschaft', 'FINANZEN', 'Finanzen', 'Wirtschaft kompakt', 'Wirtschaft Kompakt', 
                 'Finanzen kompakt', 'Finanzen Kompakt', 'wirtschaft kompakt', 'finanzen kompakt']

In [14]:
# Keep only relevant sections
data = data[data['section'].isin(keep_sections)]
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

## Titles

Titles to clean:
* 'Verlierer.': stock prices only.
* 'Gewinner.': stock prices only.
* 'Auf dem Sprung an die Börse.': quantitative info on companies that plan to go public.
* 'Aktuelle Neuemissionen.': quantitative info on new issues.
* 'Zinsen in Prozent.': quantitative info on interest rates. 
* 'Die Schlagzeile vom 1962.' ('Die Schlagzeile vom 1963.'): headlines from 1962/1963, news from the past.
* 'Leserfrage der Woche.' ('Leserfrage.'): reader's questions, have nothing to do with the Economy. Usually, these are some legal questions.
* 'Messe-Termine.': trade show dates.
* 'service splitter.' ('ServiceSplitter.', 'Service-Splitter.', 'Service Splitter.') : has nothing to do with the economy.
* 'Die Steuer-Serie auf einen Blick.': the tax series, not economy-related.
* 'WebWegWeiser.': web news.
* 'Die Titel mit den größten Kursausschlägen. Stand: Schluss.': stock prices.
* 'Wirtschaft. Vernetzt im Wohnzimmer.' ('Vernetzt im Wohnzimmer.'): advertisement of digital products.
* 'Telefonaktion.': advertisement of the possibility to ask questions over the phone about a particular topic.
* 'TICKER.' ('Ticker.'): stock prices.
* 'Heizöl-Preise aktuell.': current heating oil prices.
* 'Die Titel mit den größten Kursausschlägen. Schluss.': stock prices.
* 'Telefontarife.': phone rates (numbers).
* 'heute im sportteil.': today in sports section (irrelevant).
* 'WELT-Rangliste der deutschen Wirtschaft - Auf einen Blick: Umsatz und. Beschäftigung; Die 500 größten Unternehmen des Landes auf einen Blick: Umsatz, Branchen und Beschäftigte.': sales and employment in the largest companies(numbers).
* 'Serie im Überblick.' ('Die Serie im Überblick.'): short introduction of the series.
* 'So können Sie Ihr Portfolio optimieren.': the asset check organized by Welt.
* 'Richtig vorsorgen. So finden Sie den perfekten Riester-Vertrag.': how to find the perfect pension plan.
* 'Die Titel mit den größten Kursausschlägen. Stand: 16.00 Uhr.': stock prices.
* 'wirtschaft-bestseller.' ('Wirtschaft-Bestseller.'): book prices.
* 'Themen (Fotos).': introduction of the topics discussed in the issue.

In [15]:
title_clean = ['Verlierer.', 'Gewinner.', 'Auf dem Sprung an die Börse.', 'Aktuelle Neuemissionen.',
              'Zinsen in Prozent.', 'Die Schlagzeile vom 1962.', 'Leserfrage der Woche.',
              'Messe-Termine.', 'service splitter.', 'ServiceSplitter.', 'Service-Splitter.', 'Service Splitter.',
              'Die Steuer-Serie auf einen Blick.', 'WebWegWeiser.',
              'Die Titel mit den größten Kursausschlägen. Stand: Schluss.',
              'Wirtschaft. Vernetzt im Wohnzimmer.', 'Telefonaktion.', 'TICKER.',
              'Vernetzt im Wohnzimmer.', 'Die Schlagzeile vom 1963.', 'Heizöl-Preise aktuell.',
              'Die Titel mit den größten Kursausschlägen. Schluss.', 'Telefontarife.',
              'heute im sportteil.', 
              'WELT-Rangliste der deutschen Wirtschaft - Auf einen Blick: Umsatz und. Beschäftigung; Die 500 größten Unternehmen des Landes auf einen Blick: Umsatz, Branchen und Beschäftigte.',
              'Serie im Überblick.', 'So können Sie Ihr Portfolio optimieren.',
              'Richtig vorsorgen. So finden Sie den perfekten Riester-Vertrag.',
              'Die Serie im Überblick.', 'Leserfrage.', 'Ticker.',
              'Die Titel mit den größten Kursausschlägen. Stand: 16.00 Uhr.'
              'wirtschaft-bestseller.', 'Wirtschaft-Bestseller.', 'Themen (Fotos).']

In [16]:
data = data[~data['titles'].isin(title_clean)]
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [17]:
import collections
counter=collections.Counter(data['titles'])
print(counter.most_common(1000))

[('', 17981), ('Menschen und Märkte.', 1423), ('Hamburger Börse.', 1369), ('Gewinner und Verlierer.', 887), ('Der Tag heute.', 442), ('Markt und Meinung.', 412), ('HAMBURGER BÖRSE.', 390), ('Das jüngste Gerücht.', 348), ('Ölpreis-Ticker.', 143), ('Analysten-Barometer.', 99), ('Der Manager.', 78), ('Zur Person.', 74), ('Das Unternehmen.', 73), ('Finanz-Meldungen.', 71), ('MENSCHEN UND MÄRKTE.', 65), ('Euwax.', 56), ('WIRTSCHAFT KOMPAKT.', 49), ('Der Chef.', 42), ('Menschen Und Märkte.', 40), ('ÖLPREIS-TICKER.', 38), ('Fazit.', 37), ('markt und meinung.', 36), ('DER TAG HEUTE.', 36), ('Die Firma.', 34), ('Meldungen.', 32), ('ANALYSE.', 31), ('DAS JÜNGSTE GERÜCHT.', 29), ('Analyse.', 29), ('Gewinner - Verlierer.', 29), ('Wirtschaft und Politik.', 28), ('Im DAX-100 verzeichneten gestern bis 13.00 Uhr folgende Titel die größten. Kursausschläge (Basis: Schlusskurs Vortag).', 28), ('MARKT UND MEINUNG.', 27), ('Im HDax verzeichneten gestern bis 13.00 Uhr folgende Titel die größten. Kursausschl

## Texts

In [18]:
# remove articles that contain the following string: "HIGHLIGHT: Die Titel mit den größten Kursausschlägen."
# and have the title 'Gewinner und Verlierer.' (stock prices, no additional info)
data = data.loc[~((data['texts'].str.contains('HIGHLIGHT: Die Titel mit den größten Kursausschlägen.')) & (data['titles']=='Gewinner und Verlierer.'))]

In [19]:
# Stock prices, no additional info
data = data.loc[~((data['texts'].str.contains('Gewinner - Verlierer. Verlierer. 1.')) | (data['texts'].str.contains('Gewinner - Verlierer. Gewinner. 1.')))]

In [20]:
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [22]:
data['w_count'] = data['texts'].str.split(' ').str.len()

In [23]:
# remove articles with the number of words<50
data = data[data['w_count']>50]

In [25]:
data = data.reset_index() # reset the index of the DataFrame
del data['index'] # delete a column with an old index

In [ ]:
# save the final version
data.to_csv('cleaned_welt.csv', encoding='utf-8-sig', sep = ';')

# In case you want to use pickle to save the Dataframe

In [12]:
import pickle

with open('welt_data.pickle', 'wb') as f:
    pickle.dump(data, f, protocol=2)

In [13]:
with open('welt_data.pickle', 'rb') as f:
    data = pickle.load(f)